In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import torch
from torch import nn 
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import cv2
import json

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_ident = "bidiretional_lstm_hrnet_nturgb"
unique_iden = "epoch10_emb2048_xy"

main_dir = "D:\\FYP\\HAR-ZSL-XAI"
data_files = os.path.join(main_dir,"pyskl","HRNetSkeletons","nipun_video_dataset","PAMAP2_K10_V1","combined.pkl")
epoch_vids = os.path.join(main_dir,"epoch_vids")
models_saves = os.path.join(main_dir,"model_saves")
embeddings_save = os.path.join(main_dir,"embedding_save")
prototypes_save = os.path.join(main_dir,"prototypes")
test_vids = os.path.join(main_dir,"test_vids")
batch_size = 32

os.makedirs(embeddings_save,exist_ok=True)

In [4]:
from utils.test_utils import *
from dataset.SkeletonData.array_segment import normalize_array

In [5]:
model_params, model_config, config = get_config(f"{models_saves}/{model_ident}/{unique_iden}.pt")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\FYP\\HAR-ZSL-XAI\\model_saves/bidiretional_lstm_hrnet_nturgb/epoch10_emb2048_xy.pt'

In [ ]:
def classname_id(class_name_list):
    id2classname = {k:v for k, v in zip(list(range(len(class_name_list))),class_name_list)}
    classname2id = {v:k for k, v in id2classname.items()}
    return id2classname, classname2id

In [ ]:
 class_names = [str(x) for x in os.listdir("../data/nipun_video_dataset/PAMAP2_K10_V1")]

In [ ]:
id2clsname, clsname2id = classname_id(class_names)

In [ ]:
with open(data_files,"rb") as f0:
    filelist = pickle.load(f0)

filter_arr = []
for __f in filelist:
    if len(__f["keypoint"].shape) == 4 and __f["keypoint"].shape[0]>0:
        filter_arr.append(__f)

num_list = len(filter_arr)

In [ ]:
num_list

In [ ]:
set_fil = set()
for fil in filter_arr:
    set_fil.add(fil["frame_dir"])

set_fil,len(set_fil)

In [ ]:
class SkeletonDataset(Dataset):
    def __init__(self, file_list,class2id, transform=None,
                 target_transform=None,active_locations=[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],file_name=False, is_2d=False):
        self.file_list = file_list
        self.transform = transform
        self.class2id = class2id
        self.target_transform = target_transform
        self.active_locations = active_locations
        self.file_name = file_name
        self.is_2d = is_2d

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        a_file = self.file_list[idx]
        action_type = a_file["label"]
        coords, vidsize = a_file["keypoint"][0],a_file["original_shape"]
        coords = coords[10:60,:,:]
        coords = coords[:,self.active_locations,:]

        normalize_array(coords,vidsize)
        if self.is_2d:
            coords = coords[...,0:2]

        shape = coords.shape

        coords = torch.from_numpy(coords).float()

        coords = torch.reshape(coords, (shape[0], shape[1]*shape[2]))
        label = torch.clone(coords)

        if self.transform:
            coords = self.transform(coords)
        if self.target_transform:
            label = self.target_transform(coords)

        if self.file_name:
            return coords, label, self.class2id[action_type],a_file["original_shape"],self.file_list[idx]["frame_dir"]
        return coords, label, self.class2id[action_type],a_file["original_shape"]

In [ ]:
skel_data = SkeletonDataset(filter_arr,clsname2id,is_2d=True,file_name=True)

In [ ]:
skel_dl = DataLoader(skel_data, batch_size=32, shuffle=False)

In [ ]:
for x in skel_dl:
    print(x[0].shape)
    break

In [ ]:
class BiLSTMEncoder(nn.Module):
    def __init__(self,seq_len, input_size, hidden_size,linear_filters,embedding_size:int, num_layers = 1,bidirectional=True,dev=device):
        super(BiLSTMEncoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dev=dev
        self.num_layers = num_layers
        self.linear_filters = linear_filters
        self.embedding_size = embedding_size
        self.bidirectional = bidirectional
        self.seq_len = seq_len

        # define LSTM layer
        self.layers = []

        # add linear layers 
        for __id,layer_out in enumerate(self.linear_filters):
            if __id == 0:
                self.layers.append(nn.Linear(self.input_size, layer_out))
            else:
                self.layers.append(nn.Linear(self.linear_filters[__id-1], layer_out))

        # add lstm layer
        self.lstm = nn.LSTM(input_size = layer_out, hidden_size = self.hidden_size,
                            num_layers = self.num_layers, bidirectional=self.bidirectional,
                            batch_first=True)
        
        self.net = nn.Sequential(*self.layers)

        #add embedding out
        if bidirectional:
            self.out_linear = nn.Linear(self.hidden_size*4, self.embedding_size)
        else:
            self.out_linear = nn.Linear(self.hidden_size*2, self.embedding_size)

        
    def forward(self, x_input):
        """
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence; hidden gives the hidden state and cell state for the last element in the sequence
        """
        
        x = self.net(x_input)
        lstm_out, self.hidden = self.lstm(x)
        hidden_transformed = torch.cat(self.hidden,0)
        hidden_transformed = torch.transpose(hidden_transformed,0,1)
        hidden_transformed = torch.flatten(hidden_transformed,start_dim=1)
        
        hidden_transformed = self.out_linear(hidden_transformed)
        
        return lstm_out, hidden_transformed

    
class BiLSTMDecoder(nn.Module):
    def __init__(self,seq_len, input_size, hidden_size, linear_filters,embedding_size:int, num_layers = 1,bidirectional=True,dev=device):
        super(BiLSTMDecoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dev = dev
        self.num_layers = num_layers
        self.linear_filters = linear_filters[::-1]
        self.embedding_size = embedding_size
        self.bidirectional = bidirectional
        self.seq_len = seq_len

        if bidirectional:
            self.input_linear = nn.Linear(self.embedding_size,4*self.hidden_size)
        else:
            self.input_linear = nn.Linear(self.embedding_size,2*self.hidden_size)

        # define LSTM layer
        self.layers = []
        # add lstm
        self.lstm = nn.LSTM(input_size = self.linear_filters[0], hidden_size = self.hidden_size,
                            num_layers = self.num_layers, bidirectional=True,
                            batch_first=bidirectional)

                        
        # add linear layers 
        if bidirectional:
            self.layers.append(nn.Linear(2*hidden_size,self.linear_filters[0]))
        else:
            self.layers.append(nn.Linear(hidden_size,self.linear_filters[0]))

        for __id,layer_in in enumerate(self.linear_filters):
            if __id == len(linear_filters)-1:
                self.layers.append(nn.Linear(layer_in,self.input_size))
            else:
                self.layers.append(nn.Linear(layer_in,self.linear_filters[__id+1]))

        self.net = nn.Sequential(*self.layers)

        
        

    def forward(self,encoder_hidden):
        """
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence; hidden gives the hidden state and cell state for the last element in the sequence
        """
        
        
        hidden_shape = encoder_hidden.shape
        encoder_hidden = self.input_linear(encoder_hidden)
        
        if self.bidirectional:
            hidden = encoder_hidden.view((-1,4,self.hidden_size))
            hidden = torch.transpose(hidden,1,0)
            h1,h2,c1,c2 = torch.unbind(hidden,0)
            h,c = torch.stack((h1,h2)),torch.stack((c1,c2))
            bs = h.size()[1]
        else:
            hidden = encoder_hidden.view((-1,2,self.hidden_size))
            hidden = torch.transpose(hidden,1,0)
            h,c = torch.unbind(hidden,0)
            bs = h.size()[1]
        
        dummy_input = torch.rand((bs,self.seq_len,self.hidden_size), requires_grad=True).to(self.dev)
        
        lstm_out, self.hidden = self.lstm(dummy_input,(h,c))
        x = self.net(lstm_out)
        
        return x

class BiLSTMEncDecModel(nn.Module):
    def __init__(self,seq_len, input_size, hidden_size, linear_filters=[128,256,512],embedding_size:int=256, num_layers = 1,bidirectional=True,dev=device):
        super(BiLSTMEncDecModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dev = dev
        self.num_layers = num_layers
        self.linear_filters = linear_filters[::-1]
        self.embedding_size = embedding_size
        self.bidirectional = bidirectional
        self.batch_size = batch_size
        self.seq_len = seq_len
        
        self.encoder = BiLSTMEncoder(seq_len, input_size, hidden_size, linear_filters,embedding_size, num_layers = 1,bidirectional=True, dev=self.dev)
        self.decoder = BiLSTMDecoder(seq_len, input_size, hidden_size, linear_filters,embedding_size, num_layers = 1,bidirectional=True, dev=self.dev)
        
    def forward(self,x):
        lstm_out,embedding = self.encoder(x)
        decoder_out = self.decoder(embedding)
        
        return decoder_out, embedding  


In [ ]:
config

In [ ]:
bilstm_model = BiLSTMEncDecModel(
    seq_len=config["model"]["seq_len"],
    input_size=config["model"]["input_size"],
    hidden_size=config["model"]["hidden_size"],
    linear_filters=config["model"]["linear_filters"],
    embedding_size=config["model"]["embedding_size"],
    num_layers = config["model"]["num_layers"],
    bidirectional=config["model"]["bidirectional"],
    dev=config["model"]["dev"])

In [ ]:
bilstm_model.to(device)

In [ ]:
bilstm_model.load_state_dict(model_params)

In [ ]:
lstm_out, embedding = bilstm_model.encoder(torch.randn((32,50,24)).to(device))

In [ ]:

embedding.shape

In [ ]:
decoder_out = bilstm_model.decoder(embedding)

In [ ]:
model_out,embedding = bilstm_model(torch.randn((1,50,24)).to(device))

In [ ]:
model_out.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mapping_l = [
        [15, 13], [13, 11], [11, 5],
        [12, 14], [14, 16], [12, 6],
        [3, 1],[1, 2],[1, 0],[0, 2],[2,4],
        [9, 7], [7,5], [5, 6],
        [6, 8], [8, 10],
        ]
#mapping_l = []

from dataset.SkeletonData.visualize import *
from utils.train_utils import *

In [ ]:
def model_predict(model,input_ten:torch.Tensor,dev):
    input_ten = input_ten.to(dev)
    _, embed_ten = model.encoder(input_ten)
    output_ten = model.decoder(embed_ten)

    return embed_ten, output_ten

def model_saves(loc,embed_ten,output_ten,detail_ten):
    np.savez(loc,embedding=embed_ten,output=output_ten,file_list=detail_ten)


In [ ]:
file_map = []
embed_map = []
output_map = []
for data_point in skel_dl:
    arrays, labels, class_nms ,video_shape,file_details = data_point
    embeddings, outputs = model_predict(bilstm_model,arrays,device)
    embeddings = embeddings.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    for e,o,d,l,cln in zip(embeddings,outputs,file_details,labels,class_nms):
        file_map.append([d])
        embed_map.append(e)
        output_map.append(o)

file_map = np.asarray(file_map)
embed_map = np.asarray(embed_map)
output_map = np.asarray(output_map)

os.makedirs(f"{embeddings_save}/{model_ident}/{unique_iden}",exist_ok=True)
model_saves(f"{embeddings_save}/{model_ident}/{unique_iden}/without_pretrain_on_local.npz",embed_map,output_map,file_map)

## Visualizations

In [ ]:
import hypertools as hyp
import pandas as pd
import re

In [ ]:
output_data = np.load(f"{embeddings_save}/{model_ident}/{unique_iden}/without_pretrain_on_local.npz")

embed_arr,out_arr,file_arr = output_data["embedding"],output_data["output"],output_data["file_list"]

In [ ]:
embed_arr.shape,out_arr.shape

In [ ]:
file_arr,np.unique(file_arr,return_counts=True)

In [ ]:
import random

def draw_heatmaps(arr_list,nrows=2,ncols=2):
    ran_list = random.sample(arr_list,ncols*nrows)
    fig, ax = plt.subplots(nrows=nrows,ncols=ncols, sharex=True)
    for i in range(nrows):
        for j in range(ncols):
            #print(i*ncols+j,len(ran_list))
            ax[i,j].imshow(ran_list[i*ncols+j].detach().cpu().numpy()[np.newaxis,:], cmap="plasma", aspect="auto")

    plt.tight_layout()
    plt.show()

In [ ]:
embed_arr.shape

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax.imshow(embed_arr, cmap="plasma", aspect="auto")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax.imshow(embed_arr[0][np.newaxis,:], cmap="plasma", aspect="auto")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
ax.imshow(embed_arr[20][np.newaxis,:], cmap="plasma", aspect="auto")

plt.tight_layout()
plt.show()

In [ ]:
class_names = [re.findall(r'[a-z]+',x[0])[0] for x in file_arr.tolist()]
class_names

In [ ]:
map_acr_full = {}
for fol in os.listdir("../data/nipun_video_dataset/PAMAP2_K10_V1"):
    file_name = os.listdir(f"../data/nipun_video_dataset/PAMAP2_K10_V1/{fol}")[0]
    map_acr_full[re.findall(r'[a-z]+',file_name)[0]] = fol

In [ ]:
class_names = [map_acr_full[x] for x in class_names]

In [ ]:
df = pd.DataFrame(data=embed_arr)

In [ ]:
hyp.plot(df, '.', reduce='UMAP', hue=class_names, ndims=3)

In [ ]:
prototypes = {}
for k,v in map_acr_full.items():
    prototypes[v] = np.mean(embed_arr[v == np.asarray(class_names)],axis=0)

In [ ]:
prototypes

In [ ]:
df = pd.DataFrame.from_dict(prototypes,orient="index")

In [ ]:
hyp.plot(df, '.', reduce='UMAP', hue=df.index, ndims=3)

In [ ]:
os.makedirs(f"{prototypes_save}/{model_ident}/{unique_iden}",exist_ok=True)

with open(f"{prototypes_save}/{model_ident}/{unique_iden}/without_pretrain_on_local.pkl","wb") as f0:
    pickle.dump(prototypes,f0)